In [84]:
import v20
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

API_KEY = config['OANDA2']['API_KEY'] 
ACCOUNT_ID = config['OANDA2']['ACCOUNT_ID']
HOSTNAME   = "api-fxpractice.oanda.com"
STREAMHOST = "stream-fxpractice.oanda.com"
key = f'Bearer {API_KEY}'

# contexts
ctx = v20.Context(hostname=HOSTNAME, token=key)
ctx.set_header(key='Authorization', value=key)

In [85]:
ac = ctx.account.summary(ACCOUNT_ID).get('account')

In [86]:
ti = ['EUR_USD', 'EUR_CAD', 'EUR_NZD', 'EUR_CHF', 'EUR_JPY', 'EUR_AUD', 'EUR_GBP', 'GBP_USD', 'GBP_CAD', 'GBP_JPY', 'GBP_AUD',
      'AUD_USD', 'AUD_CAD', 'AUD_NZD', 'AUD_JPY', 'NZD_USD', 'NZD_JPY', 'USD_CHF', 'USD_CAD', 'USD_JPY']
ti_csv = ','.join(ti)

In [87]:
insts = ctx.account.instruments(ACCOUNT_ID, instruments=ti_csv).get('instruments')

In [88]:
print(insts[0])

name: EUR_USD
type: CURRENCY
displayName: EUR/USD
pipLocation: -4
displayPrecision: 5
tradeUnitsPrecision: 0
minimumTradeSize: 1.0
maximumTrailingStopDistance: 1.0
minimumTrailingStopDistance: 0.0005
maximumPositionSize: 0.0
maximumOrderUnits: 100000000.0
marginRate: 0.0333


In [89]:
insts[0].displayPrecision

5

In [95]:
inst = insts[18]
u = -108
units = f'{u:.{inst.tradeUnitsPrecision}f}'

gp_sl = 25 * pow(10, inst.pipLocation)
gp_ts = 35 * pow(10, inst.pipLocation)
gp_tp = 40 * pow(10, inst.pipLocation)

sl_on_fill = dict(
    timeInForce='GTC', 
    # price=f'{sl_price:.{insts.displayPrecision}f}',
    distance=f'{gp_sl:.{inst.displayPrecision}f}')

tp_on_fill = dict(
    timeInForce='GTC', 
    # price=f'{tp_price:.{inst.displayPrecision}f}',
    distance=f'{gp_tp:.{inst.displayPrecision}f}')

ts_on_fill = dict(
    timeInForce='GTC', 
    distance=f'{gp_ts:.{inst.displayPrecision}f}')

order = dict(
    type='MARKET',
    instrument=inst.name,
    units=units,
    takeProfitOnFill=tp_on_fill,
    stopLossOnFill=sl_on_fill,
    trailingStopLossOnFill=ts_on_fill)

print(order)
response = ctx.order.market(ACCOUNT_ID, **order)
print(response)

for b in response.body:
    print(response.get(b))
    print('')

{'type': 'MARKET', 'instrument': 'USD_CAD', 'units': '-108', 'takeProfitOnFill': {'timeInForce': 'GTC', 'distance': '0.00400'}, 'stopLossOnFill': {'timeInForce': 'GTC', 'distance': '0.00250'}, 'trailingStopLossOnFill': {'timeInForce': 'GTC', 'distance': '0.00350'}}
Method = POST
Path = https://api-fxpractice.oanda.com:443/v3/accounts/101-012-20737061-001/orders
Status = 201
Reason = Created
Content-Type = application/json

id: '101133'
time: '2022-05-29T21:11:58.793541603Z'
userID: 20737061
accountID: 101-012-20737061-001
batchID: '101133'
requestID: '60979981765376071'
type: MARKET_ORDER
instrument: USD_CAD
units: -108.0
timeInForce: FOK
positionFill: DEFAULT
reason: CLIENT_ORDER
takeProfitOnFill:
  timeInForce: GTC
stopLossOnFill:
  distance: 0.0025
  timeInForce: GTC
trailingStopLossOnFill:
  distance: 0.0035
  timeInForce: GTC

id: '101134'
time: '2022-05-29T21:11:58.793541603Z'
userID: 20737061
accountID: 101-012-20737061-001
batchID: '101133'
requestID: '60979981765376071'
type: 

In [ ]:
olp=ctx.order.list_pending(ACCOUNT_ID)